## The Equations of motion for 3-bodies
---
 We know $F = \frac{G m_i m_j}{r^2}$,  where $r = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}$

and for $r = \sqrt{(x_i - x_j)^2 + (y_i - y_j)^2}$, we have: $\vec{r_i} = x_i\hat{x}+y_i\hat{y}$, ($i = 1,2,3,...$)

and we also know: $\vec{F} = m\vec{a}$ 

so, for mass1 and mass2, we know the force between them:

in x direction:

$\ddot{x_1} =  \frac{G m_2}{(x_1 - x_2)^2 + (y_1 - y_2)^2} \frac{x_2 - x_1}{\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2)}}$ 

in y direction:

$\ddot{y_1} =  \frac{G m_2}{(x_1 - x_2)^2 + (y_1 - y_2)^2} \frac{y_2 - y_1}{\sqrt{(x_1 - x_2)^2 + (y_1 - y_2)^2)}}$ 

in same way. we  could get: 

for $m_i$

$\ddot{x_i} = \sum_{j=1}^{n} \frac{G m_j}{[(x_i - x_j)^2 + (y_i - y_j)^2]^\frac{3}{2}} (x_j - x_i)$ 

$\ddot{y_i} = \sum_{j=1}^{n} \frac{G m_j}{[(x_i - x_j)^2 + (y_i - y_j)^2]^\frac{3}{2}} (y_j - y_i)$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint, solve_ivp
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
# Change the common font size
font_size = 14
plt.rcParams.update({'font.size': font_size})

In [ ]:
class GravitationalOrbist:
    """
    This class implements the parameters and Larange's equations for two particles orbiting
    under gravitational attraction.
    
    Parameters
    ----------
    m_1 : 
        type:float
        Physical meaning: mass
    m_2 : 
        type:float
        Physical meaning: mass
    m_3 : 
        type:float
        Physical meaning: mass
    G : 
        type:float
        Physical meaning: G

    Methods
    ----------
    dy_dt(t, y)
        This function returns the right-hand side of the diffeq: 
        [dy/dt d^2y/dt^2]
    """
    
    def __init__(self, m_1=1, m_2=1, m_3 = 1, G=1):
        self.m_1 = m_1
        self.m_2 = m_2
        self.m_3 = m_3
        self.G = G
        
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        
        Parameters
        ----------
        t : 
            type: float
            Physical meaning: time 
        y : 
            type: float
            Physical meaning: 12-component vector with 
            y[0] = x1(t), y[1] = dx1(t)/dt, 
            y[2] = y1(t), y[3] = dy1(t)/dt, 
            y[4] = x2(t), y[5] = dx2(t)/dt, 
            y[6] = y2(t), y[7] = dy2(t)/dt, 
            y[8] = x3(t), y[9] = dx3(t)/dt, 
            y[10] = y3(t), y[11] = dy3(t)/dt, 
            
        """
        # Denominator that shows up in all of the equations   
        x_dot_1 = y[1]
        y_dot_1 = y[3]
        x_dot_2 = y[5]
        y_dot_2 = y[7]
        x_dot_3 = y[9]
        y_dot_3 = y[11]
        
        r_12 = ((y[4]-y[0])**2 + (y[6]-y[2])**2)**(3/2) #r for mass 1 and 2
        r_13 = ((y[8]-y[0])**2 + (y[10]-y[2])**2)**(3/2) #r for mass 1 and 3
        r_23 = ((y[8]-y[4])**2 + (y[10]-y[6])**2)**(3/2) #r for mass 2 and 3
 
        x_dot_dot_1 = self.G*self.m_3*(y[8]-y[0])/(r_13) + self.G*self.m_2*(y[4]-y[0])/(r_12)
        y_dot_dot_1 = self.G*self.m_3*(y[10]-y[2])/(r_13) + self.G*self.m_2*(y[6]-y[2])/(r_12)
        x_dot_dot_2 = self.G*self.m_1*(y[0]-y[4])/(r_12)+self.G*self.m_3*(y[8]-y[4])/(r_23)
        y_dot_dot_2 = self.G*self.m_1*(y[2]-y[6])/(r_12)+self.G*self.m_3*(y[10]-y[6])/(r_23)
        x_dot_dot_3 = self.G*self.m_2*(y[4]-y[8])/(r_23) + self.G*self.m_1*(y[0]-y[8])/(r_13)
        y_dot_dot_3 = self.G*self.m_1*(y[2]-y[10])/(r_13)+self.G*self.m_2*(y[6]-y[10])/(r_23)
               
        
        return x_dot_1, x_dot_dot_1, y_dot_1, y_dot_dot_1, x_dot_2, x_dot_dot_2, y_dot_2, y_dot_dot_2, x_dot_3, x_dot_dot_3, y_dot_3, y_dot_dot_3
    
    
        
    def solve_ode(self, t_pts, x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0, x_3_0, x_dot_3_0, y_3_0, y_dot_3_0,
                  method='RK23',
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Specify smaller abserr and relerr to get more precision.
        
        """
        
        y = [x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0, x_3_0, x_dot_3_0, y_3_0, y_dot_3_0]  
        solution = solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), 
                             y, t_eval=t_pts, method=method, 
                             atol=abserr, rtol=relerr)
        
        x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2, x_3, x_dot_3, y_3, y_dot_3 = solution.y
        
        return x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2, x_3, x_dot_3, y_3, y_dot_3

In [ ]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

# Setting initial value
m_1 = 10.
m_2 = 15.
m_3 = 10.
G = 1.

x_2_0, x_dot_2_0 = 2, 0.5
y_2_0, y_dot_2_0 = 1., 2
x_3_0, x_dot_3_0 = -2, 0
y_3_0, y_dot_3_0 = 0., -1.2
x_1_0, x_dot_1_0 = 0., -(x_dot_2_0*m_2+x_dot_3_0*m_3)/m_1
y_1_0, y_dot_1_0 = 0., -(y_dot_2_0*m_2+y_dot_3_0*m_3)/m_1

t_start = 0.
t_end = 30.
delta_t = 0.01
t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

o1 = GravitationalOrbist(m_1 = m_1, m_2 = m_2, m_3 = m_3, G = G)

x_1, x_dot_1, y_1, y_dot_1, x_2, x_dot_2, y_2, y_dot_2, x_3, x_dot_3, y_3, y_dot_3 = o1.solve_ode(t_pts, x_1_0, x_dot_1_0, y_1_0, y_dot_1_0, x_2_0, x_dot_2_0, y_2_0, y_dot_2_0,x_3_0, x_dot_3_0, y_3_0, y_dot_3_0)

Everything is ready, now start making the graphs

In [ ]:
fig = plt.figure(figsize=(8,8))

ax_4a = fig.add_subplot(1,1,1)
ax_4a.plot(x_1, y_1, color='red', label='m_1')
ax_4a.plot(x_2, y_2, color='orange', label='m_2')
ax_4a.plot(x_3, y_3, color='blue', label='m_3')

ax_4a.set_ylim(-6,6)
ax_4a.set_xlim(-6,6)

ax_4a.set_title('Orbit: Three Body')
ax_4a.set_xlabel('x')
ax_4a.set_ylabel('y')

ax_4a.legend()

In [ ]:
x_min = min([min(x_1),min(x_2),min(x_3)]) 
y_min = min([min(y_1),min(y_2),max(x_3)]) 

x_max = max([max(x_1),max(x_2),min(x_3)]) 
y_max = max([max(y_1),max(y_2),max(x_3)]) 

gap = 5
fig_animate = plt.figure(figsize=(6,6), num='Orbit: Three Body')
ax_animate = fig_animate.add_subplot(1,1,1)
ax_animate.set_xlim(x_min-gap, x_max+gap)
ax_animate.set_ylim(y_min-gap, y_max+gap)

# By assigning the first return from plot to line_anim, we can later change
#  the values in the line.
graph_animate_1, = ax_animate.plot(x_1[0], y_1[0], '*', markersize=5, color='red',)
graph_animate_2, = ax_animate.plot(x_2[0], y_2[0],  '*', markersize=5, color='orange')
graph_animate_3, = ax_animate.plot(x_3[0], y_3[0],  '*', markersize=5, color='blue')


In [ ]:
def animate_orbit(i):
    """
    This function called by FuncAnimation to create each frame,
    numbered by i.  
    So each i corresponds to a point in the t_pts(array) with index i. 
    """

    graph_animate_1.set_data(x_1[i], y_1[i])  # overwrite graph_animate with new points
    graph_animate_2.set_data(x_2[i], y_2[i])  
    graph_animate_3.set_data(x_3[i], y_3[i])  
    return (graph_animate_1, graph_animate_2, graph_animate_3)   


In [ ]:
frame_interval = 5.  # time between frames
frame_number = 2000 # number of frames (ie. index of t_pts)
anim = animation.FuncAnimation(fig_animate, 
                               animate_orbit, 
                               init_func=None,
                               frames=frame_number, 
                               interval=frame_interval, 
                               blit=True,
                               repeat=False)

#fig.show()


In [ ]:
HTML(anim.to_jshtml())

### Comment

The three-body system is a complex system, and I spent a long time looking for a seemingly "good" orbit, because this system is very sensitive to any perturbations

But this is only the intake of three objects; in fact, if we study the laws of motion of electrons in some solid, we will be dealing with $10^{23}$ electrons which is completely unmanageable.

But physics did not stop there, and with the famous phrase *More is Different*, physics has given rise to a completely different field (condensed matter physics) and a different philosophy (Emergentism).